In [1]:
using PyCall

In [2]:
redis = pyimport("redis")
query = pyimport("redis.commands.search.query")
r = redis.Redis(host="localhost", port=6379)

PyObject Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>

In [3]:
_docs = r.ft("b:nodes").search("*").docs
_docs

6-element Vector{PyObject}:
 PyObject Document {'id': 'b:n:eef834d90f769df66eeefad4f800e693fe4ff413', 'payload': None, 'searchable': '00000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000

In [4]:
_query = (
    query.Query("*").return_fields("sha1", "labels", "searchable").paging(0, 100)
)

_docs = r.ft("b:nodes").search(_query).docs
q_vector = _docs[1]["searchable"]

PyObject '000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000000

In [5]:
stmt = raw"*=>[KNN 100 @searchable $vec as score]"
_query = (
    query.Query(stmt).sort_by("score").return_fields("sha1", "labels", "score").paging(0, 100).dialect(2)
)

query_params = Dict("vec" => q_vector)

r.ft("b:nodes").search(_query, query_params).docs

6-element Vector{PyObject}:
 PyObject Document {'id': 'b:n:eef834d90f769df66eeefad4f800e693fe4ff413', 'payload': None, 'score': '0', 'sha1': 'eef834d90f769df66eeefad4f800e693fe4ff413', 'labels': 'node1'}
 PyObject Document {'id': 'b:n:ab431ec0a00c5a9227c65b00055cdd9d77ee51da', 'payload': None, 'score': '5.36517749986e-14', 'sha1': 'ab431ec0a00c5a9227c65b00055cdd9d77ee51da', 'labels': 'union'}
 PyObject Document {'id': 'b:n:64d3b49ded489e3270fc584dd468be01d0f3d0b2', 'payload': None, 'score': '1.76455017264e-09', 'sha1': '64d3b49ded489e3270fc584dd468be01d0f3d0b2', 'labels': 'node2'}
 PyObject Document {'id': 'b:n:22c64bad7a5f32594770a6c52413dc193f5d0a62', 'payload': None, 'score': '3.52910012325e-09', 'sha1': '22c64bad7a5f32594770a6c52413dc193f5d0a62', 'labels': 'node4'}
 PyObject Document {'id': 'b:n:14a22df49147b1d019210e05e4c5925bf22a6e62', 'payload': None, 'score': '1.41164040457e-08', 'sha1': '14a22df49147b1d019210e05e4c5925bf22a6e62', 'labels': 'node3'}
 PyObject Document {'id': 'b

# References
1. https://juliapy.github.io/PythonCall.jl/stable/pythoncall/
2. https://redis.io/docs/latest/develop/interact/search-and-query/advanced-concepts/tags/ 